In [8]:
# FASTF1 Setup & 2023 Data Export
import fastf1
from fastf1 import get_event_schedule, get_session
import pandas as pd
import os
import logging


In [9]:
# --- Logging Setup ---
logging.basicConfig(filename='fastf1_data_export.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


In [10]:
# --- FastF1 Cache ---
cache_dir = '../.fastf1_cache'
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)


In [11]:
# --- Output Directory ---
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

# --- Load 2023 Event Schedule ---
schedule = get_event_schedule(2023, include_testing=False)
race_events = schedule[schedule['EventFormat'] == 'conventional']

all_races = []

In [12]:
# --- Tyre Compound Mapping (optional) ---
compound_map = {
    'SOFT': 'Soft',
    'MEDIUM': 'Medium',
    'HARD': 'Hard',
    'INTERMEDIATE': 'Intermediate',
    'WET': 'Wet',
}

In [ ]:
# Tyre compound mapping
compound_map = {
    'SOFT': 'Soft', 'MEDIUM': 'Medium', 'HARD': 'Hard',
    'INTERMEDIATE': 'Intermediate', 'WET': 'Wet'
}

for _, row in race_events.iterrows():
    round_num = row['RoundNumber']
    gp_name = row['EventName'].lower().replace(" ", "_")

    try:
        session = get_session(2023, round_num, 'R')
        session.load()
    except Exception as e:
        logging.error(f"[LOAD FAIL] {gp_name}: {e}")
        print(f"❌ Failed to load session for {gp_name} — {e}")
        continue

    try:
        laps = session.laps.reset_index(drop=True)

        selected_cols = [
            'Driver', 'Team', 'LapNumber', 'LapTime',
            'Sector1Time', 'Sector2Time', 'Sector3Time',
            'Compound', 'TyreLife', 'Stint',
            'PitInTime', 'PitOutTime', 'TrackStatus', 'IsAccurate', 'Time'
        ]
        lap_data = laps[selected_cols].copy()

        # Time conversions
        for col in ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']:
            lap_data[f'{col}Seconds'] = lap_data[col].dt.total_seconds()

        lap_data['LapStartTime'] = lap_data['Time'].dt.total_seconds()

        # === WEATHER MERGE ===
        weather = session.weather_data.rename(columns={'Time': 'WeatherTime'})
        weather['WeatherTime'] = weather['WeatherTime'].dt.total_seconds()

        lap_data = pd.merge_asof(
            lap_data.sort_values('LapStartTime'),
            weather.sort_values('WeatherTime'),
            left_on='LapStartTime',
            right_on='WeatherTime',
            direction='nearest'
        )

        # Flags
        lap_data['IsWetLap'] = lap_data['Rainfall'] > 0.1
        lap_data['IsDryLap'] = lap_data['Rainfall'] <= 0.1

        # Safe cleaning and type enforcement
        lap_data['TrackStatus'] = lap_data['TrackStatus'].astype(str)
        lap_data['IsAccurate'] = lap_data['IsAccurate'].astype(bool)
        lap_data['LapNumber'] = lap_data['LapNumber'].fillna(0).astype(int)
        lap_data['TyreLife'] = lap_data['TyreLife'].fillna(0).astype(int)
        lap_data['Stint'] = lap_data['Stint'].fillna(0).astype(int)

        # Continue processing
        lap_data.dropna(subset=[
            'LapTimeSeconds', 'Sector1TimeSeconds',
            'Sector2TimeSeconds', 'Sector3TimeSeconds', 'Compound'
        ], inplace=True)

        lap_data['Compound'] = lap_data['Compound'].str.upper().map(compound_map).fillna(lap_data['Compound'])
        lap_data['PitLap'] = lap_data.apply(
            lambda row: row['LapNumber'] if pd.notna(row['PitInTime']) else None, axis=1
        )
        lap_data['PitDuration'] = (lap_data['PitOutTime'] - lap_data['PitInTime']).dt.total_seconds()
        lap_data['Sector1Pct'] = lap_data['Sector1TimeSeconds'] / lap_data['LapTimeSeconds']
        lap_data['Sector2Pct'] = lap_data['Sector2TimeSeconds'] / lap_data['LapTimeSeconds']
        lap_data['Sector3Pct'] = lap_data['Sector3TimeSeconds'] / lap_data['LapTimeSeconds']
        lap_data['BestSector'] = lap_data[['Sector1TimeSeconds', 'Sector2TimeSeconds', 'Sector3TimeSeconds']].idxmin(axis=1)
        lap_data['BestSector'] = lap_data['BestSector'].str.extract('(\d)').astype(int)
        lap_data['IsValidLap'] = (lap_data['TrackStatus'] == 'Green') & lap_data['IsAccurate']
        lap_data['GrandPrix'] = gp_name
        lap_data['SeasonYear'] = 2023

        # Stint summary
        stint_summary = lap_data.groupby(['Driver', 'Stint']).agg(
            AvgLapTime=('LapTimeSeconds', 'mean'),
            StintLength=('LapNumber', 'count')
        ).reset_index()
        lap_data = lap_data.merge(stint_summary, on=['Driver', 'Stint'], how='left')

        # Export per race
        race_csv_path = f"{output_dir}/race_summary_2023_{gp_name}.csv"
        lap_data.to_csv(race_csv_path, index=False)
        logging.info(f"[EXPORT OK] {gp_name}")
        print(f"✅ Exported: {gp_name}")

        all_races.append(lap_data)

    except Exception as e:
        logging.error(f"[PROCESS FAIL] {gp_name}: {e}")
        print(f"❌ Failed processing for {gp_name} — {e}")
        # Export per race
        race_csv_path = f"{output_dir}/race_summary_2023_{gp_name}.csv"
        lap_data.to_csv(race_csv_path, index=False)
        logging.info(f"[EXPORT OK] {gp_name}")
        print(f"✅ Exported: {gp_name}")

        all_races.append(lap_data)

    except Exception as e:
        logging.error(f"[PROCESS FAIL] {gp_name}: {e}")
        print(f"❌ Failed processing for {gp_name} — {e}")

<>:74: SyntaxWarning: invalid escape sequence '\d'
<>:74: SyntaxWarning: invalid escape sequence '\d'
/var/folders/_k/rr2k0qjs3w11xr9j_cchfb540000gn/T/ipykernel_65754/1622806169.py:74: SyntaxWarning: invalid escape sequence '\d'
  lap_data['BestSector'] = lap_data['BestSector'].str.extract('(\d)').astype(int)
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weath

✅ Exported: bahrain_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 11 completed the race distance 00:00.035000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_ap

✅ Exported: saudi_arabian_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '14', '18', '11', '4', '27', '81', '24', '22', '77', '55', '10', '31', '21', '2', '20', '63', '23', '16']
core           INFO 	Loading data for Miami Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

✅ Exported: mexico_city_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:00.001000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '31', '18', '55', '44', '63', '14', '81', '10', '23', '20', '3', '24', '2', '77', '22', '27', '4']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_d

✅ Exported: las_vegas_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '11', '4', '81', '14', '22', '44', '18', '3', '31', '10', '23', '27', '2', '24', '55', '77', '20']


✅ Exported: abu_dhabi_grand_prix

🎉 Combined export complete → ../data/processed/all_races_combined_with_sectors.csv


In [ ]:
# --- Final Combined Export ---
if all_races:
    combined_df = pd.concat(all_races, ignore_index=True)
    combined_csv_path = f"{output_dir}/all_races_combined_with_sectors.csv"
    combined_df.to_csv(combined_csv_path, index=False)
    logging.info(f"[COMBINED EXPORT] All races complete.")
    print(f"\n🎉 Combined export complete → {combined_csv_path}")
else:
    logging.warning("❌ No races processed successfully.")
    print("⚠️ No race data was processed.")